```
Let's use hashindex to solve Wordle and see if it's fast!

Here is the game hashindex played.
```

![wordle game](wordle.png)

In [1]:
# note: 469 us and 4.21ms were the timings of Frozen and Mutable respectively, prior to the rewrite.
# wonder how crappy they'll be now.

# hey nice!! Mutable is actually a little faster for whatever reason. 3.12ms now.
# Gotta write Frozen still but, you know. Yay!

In [2]:
import string
from functools import partial
from hashindex import HashIndex
from words import all_wordle_words  # contains all 5-letter Wordle words as tuples of (word, frequency in English)

In [3]:
# Generate 5 functions to fetch the letter at each position in the word.
def char_at(pos, w):
    return w[0][pos]

def make_char_at_function(pos):
    p = partial(char_at, pos)
    p.__name__ = f'at_{pos}'
    return p

char_at = {pos: make_char_at_function(pos) for pos in range(5)}

In [4]:
# Generate 26 functions that count the occurrences of the specific letter.

def char_count(c, w):
    return w[0].count(c)

def make_char_count_function(letter):
    p = partial(char_count, letter)
    p.__name__ = f'has_{letter}'
    return p

char_count = {letter: make_char_count_function(letter) for letter in string.ascii_uppercase}

In [5]:
hi = HashIndex(all_wordle_words, on=list(char_at.values()) + list(char_count.values()))
#  hi.freeze()

In [6]:
ws = hi.find(
    match = {
    },
    exclude = {
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[('ABOUT', 1226734006),
 ('OTHER', 978481319),
 ('WHICH', 810514085),
 ('THEIR', 782849411),
 ('THERE', 701170205),
 ('FIRST', 578161543),
 ('WOULD', 572644147),
 ('THESE', 541003982),
 ('CLICK', 536746424),
 ('PRICE', 501651226)]

In [7]:
ws = hi.find(
    match = {
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[('STUDY', 152978354),
 ('UNTIL', 113090086),
 ('STUFF', 81444510),
 ('QUITE', 58777731),
 ('UNITS', 56512308),
 ('SUITE', 44829675),
 ('TRUTH', 35345925),
 ('TRUCK', 32136249),
 ('TRULY', 21693553),
 ('DUTCH', 19000979)]

In [8]:
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[('HUTCH', 1373107),
 ('CUTER', 152750),
 ('MUTCH', 106606),
 ('KUTCH', 67973),
 ('TUTEE', 17303),
 ('MUTER', 6358),
 ('CUTCH', 6358)]

In [9]:
%%timeit -n 25 -r 5 
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)

3.18 ms ± 83.9 µs per loop (mean ± std. dev. of 5 runs, 25 loops each)


```
Compare: find words by list comprehension
    
A frozen HashIndex is 10x faster or so. Not too shabby for such a small dataset!
```

In [10]:
%%timeit -n 25 -r 5 
z = [w for w in all_wordle_words if 
    w[0].count('S') == 0 and
    w[0].count('A') == 0 and
    w[0].count('B') == 0 and
    w[0].count('O') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('D') == 0 and
    w[0].count('N') == 0 and
    w[0].count('I') == 0 and
    w[0].count('L') == 0 and
    w[0][2] == 'T' and
    w[0].count('U') != 0 and
    w[0].count('T') != 0 and
    w[0][0] != 'U' and
    w[0][1] != 'T' and
    w[0][2] != 'U' and
    w[0][3] != 'U' and
    w[0][4] != 'T' 
]

4.07 ms ± 87.3 µs per loop (mean ± std. dev. of 5 runs, 25 loops each)


In [11]:
z = [w for w in all_wordle_words if 
    w[0].count('S') == 0 and
    w[0].count('A') == 0 and
    w[0].count('B') == 0 and
    w[0].count('O') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('D') == 0 and
    w[0].count('N') == 0 and
    w[0].count('I') == 0 and
    w[0].count('L') == 0 and
    w[0][2] == 'T' and
    w[0].count('U') != 0 and
    w[0].count('T') != 0 and
    w[0][0] != 'U' and
    w[0][1] != 'T' and
    w[0][2] != 'U' and
    w[0][3] != 'U' and
    w[0][4] != 'T' 
]
z

[('HUTCH', 1373107),
 ('CUTER', 152750),
 ('MUTCH', 106606),
 ('KUTCH', 67973),
 ('TUTEE', 17303),
 ('MUTER', 6358),
 ('CUTCH', 6358)]